In [1]:
import os # for accessing the file system
from itertools import product # for efficient looping process

import pandas as pd # for data analysis

# for deep learning process
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# Find the count of the files in the datasets directory

In [59]:
set_list = ['test_set', 'training_set']
cats_dogs = ['cats', 'dogs']
folderpath = [os.path.abspath(f'datasets/{sets}/{animals}') for sets, animals in product(set_list, cats_dogs)]

count = sum(len(os.listdir(folderpath[i])) for i in range(len(folderpath)))
print(f"Total images found: {count} files")

Total images found: 10028 files
